In [1]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import create_ribasim_lumping_network

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")
network_name = "tki_zutphen_berkel"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", f"{network_name}_afw_eenheden.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = create_ribasim_lumping_network(
    name=network_name, 
    dhydro_dir=dhydro_dir,
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [ ]:
# Add simulation sets for winter, summer, etc.
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="6D"),
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    edges=False,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        'kst_ST80830045', # weir tussen uitstroompunten Zutphen
    ], 
    edge_ids_to_include=[],
    node_ids_to_include=[
        419, # voorbeeld splitsing
        1784, 2542 # splitsing afleidingskanaal
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    'sto_AE80770024': 'ManningResistance', 
    'kdu_DR80740070': 'ManningResistance',
    # duikers voor wijk Zuidwijken
    'kdu_DR80940046': 'ManningResistance', 
    'kdu_DR80950043': 'ManningResistance', 
    'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(f"{results_dir}/{network.name}")

Function for writing and reading excel with structures

In [ ]:
def write_structures_to_excel(network):
    """ export all structures and splitnode info to excel file with seperate sheet per structure type
     input: network with structure gdfs, splitnodes, split node type conversion tables """

    from typing import Dict
    list_gdfs = [
        # network.stations_gdf,
        network.pumps_gdf,
        network.weirs_gdf,
        network.orifices_gdf,
        network.bridges_gdf,
        network.culverts_gdf,
        network.uniweirs_gdf
    ]
    structures = pd.DataFrame(
        columns=['mesh1d_node_id', 'mesh1d_nEdges', 'geometry', 'object_type'],
    )

    if network.split_nodes is not None:
        splitnodes = network.split_nodes.copy()
        # splitnodes['splitnode']='yes'

    with pd.ExcelWriter(f"{results_dir}/{network.name}/structures.xlsx") as writer:  
        for gdf in list_gdfs:
            if gdf is not None:
                # merge structure gdf with splitnode
                if network.split_nodes is not None:
                    if network.split_node_type_conversion is not None:
                        # voeg conversie tabel toe
                        split_node_type_conversion = network.split_node_type_conversion
                        if isinstance(split_node_type_conversion, Dict):
                            for key, value in split_node_type_conversion.items():
                                splitnodes['type'] = splitnodes['split_type'].replace(split_node_type_conversion)
                        
                        split_node_id_conversion = network.split_node_id_conversion
                        if isinstance(split_node_id_conversion, Dict):
                            for key, value in split_node_id_conversion.items():
                                if len(splitnodes[splitnodes['mesh1d_node_id'] == key]) == 0:
                                    print(f" * split_node type conversion id={key} (type={value}) does not exist")
                                splitnodes.loc[splitnodes['mesh1d_node_id'] == key, 'type'] = value
                    # merge structures with splitnodes
                    structure = gdf.merge(splitnodes, left_on='mesh1d_node_id', right_on='mesh1d_node_id', how='left', suffixes=('', '_spl'),indicator=True)
                    structure['use_splitnode'] = structure['_merge'].map({'both': 'yes', 'left_only': 'no'})
                else:
                    structure=gdf.copy()
                    
                # save structures in excelfile, with seperate sheet per structuretype 
                if not structure.empty:
                    struct_name=structure['object_type'][0]
                    print(f'write {struct_name} to excel')
                    structure = structure[structure.columns & ['mesh1d_node_id', 'projection_x', 'projection_y', 'object_type','use_splitnode','status','splitnode','type']]
                    structure.to_excel(writer, sheet_name=struct_name)

    

In [ ]:
def read_structures_from_excel(excel_path):
    """ import all structure ids from excelfile 
    and output:
    - dictionary with excel data
    - list of structures to include as splitnode 
    - dictionary with structure ids and splitnode conversion type """
    
    structures_excel = pd.read_excel(excel_path, sheet_name=None) #sheet_name None to read all sheets as dictionary

    structures_ids_to_include_as_splitnode = []
    split_node_id_conversion = dict()

    for key in structures_excel:
        structure_name = key
        structure_df = structures_excel.get(structure_name)
        print(f'read sheet {key}')
        # structure_df = structures_excel.get('pump')
        
        # filter the structures to use as splitnode and add to list structures_ids_to_include_as_splitnode
        if 'use_splitnode' not in structure_df.columns:
            print(f'- no column named use_splitnode found in sheet {key}')
            
        else:
            structure_splitnodes = structure_df.loc[structure_df['use_splitnode'] == 'yes' ] 
            structure_ids = list(structure_splitnodes['mesh1d_node_id'])
            structures_ids_to_include_as_splitnode = structures_ids_to_include_as_splitnode + structure_ids

            if 'type' not in structure_df.columns:
                print(f'- no column named type found in sheet {key}')
            else:
                # convert to dictionary for split_node_id_conversion
                structure_dict = dict(zip(structure_splitnodes['mesh1d_node_id'], structure_splitnodes['type']))
                split_node_id_conversion.update(structure_dict)

            
    return structures_excel, structures_ids_to_include_as_splitnode, split_node_id_conversion


In [ ]:
write_structures_to_excel(network)

In [ ]:
excel_path = f"{results_dir}/{network.name}/structures_input.xlsx"
structures_ids_to_include_as_splitnode, split_node_id_conversion  = read_structures_from_excel(excel_path)